
# **1.Dataset Preparation**

Datasets: Use publicly available datasets such as FaceForensics++ or Celeb-DF.

Preprocessing:
- Extract frames from videos (e.g., 10 frames per video).
- Resize images to match the Xception model’s input size (299x299 pixels).
- Normalize pixel values to the range [-1, 1] (as required by Xception).
- Label frames as real or fake.


In [ ]:


import os
import numpy as np
import tensorflow as tf

class DatasetPreparation:
    def __init__(self, input_shape=(299, 299, 3), seq_length=10):
        self.input_shape = input_shape
        self.seq_length = seq_length

    def preprocess_image(self, img_path):
        """Load, resize, and normalize an image."""
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, self.input_shape[:2])
        img = tf.keras.applications.xception.preprocess_input(img)  # Normalize to [-1, 1]
        return img

    def create_dataset(self, directory, batch_size=16):
        """Create a TensorFlow dataset from images in a directory."""
        paths = []
        labels = []

        for class_name in ['real', 'fake']:
            class_dir = os.path.join(directory, class_name)
            if not os.path.exists(class_dir):
                raise ValueError(f"Directory not found: {class_dir}")

            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith('.png'):
                    paths.append(os.path.join(class_dir, img_name))
                    labels.append(1.0 if class_name == 'real' else 0.0)

        dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
        dataset = dataset.map(
            lambda path, label: (self.preprocess_image(path), label),
            num_parallel_calls=tf.data.AUTOTUNE
        )
        dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset




# **2. Model Architecture**

MADNet combines:
- EfficientNetB0 for spatial feature extraction.
- LSTM for temporal sequence analysis.
- Dense layers for final classification.

In [ ]:

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, LSTM, Dropout, TimeDistributed, Concatenate, BatchNormalization
from tensorflow.keras.models import Model

class MADNet:
    def __init__(self, input_shape=(299, 299, 3), seq_length=10):
        self.input_shape = input_shape
        self.seq_length = seq_length
        self.model = self._build_model()

    def _build_model(self):
        """Build the MADNet architecture."""
        # Frame-level input
        frame_input = Input(shape=self.input_shape, name="Frame_Input")
        base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=self.input_shape)
        spatial_features = GlobalAveragePooling2D()(base_model(frame_input))

        # Sequence-level input
        sequence_input = Input(shape=(self.seq_length, *self.input_shape), name="Sequence_Input")
        time_distributed = TimeDistributed(base_model)(sequence_input)
        temporal_features = LSTM(256, return_sequences=False)(time_distributed)

        # Combine features
        combined = Concatenate()([spatial_features, temporal_features])
        x = BatchNormalization()(combined)
        x = Dense(256, activation="relu")(x)
        x = Dropout(0.5)(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.3)(x)
        output = Dense(1, activation="sigmoid", name="Output")(x)

        return Model(inputs=[frame_input, sequence_input], outputs=output)

# **3. Model Training**

Train MADNet using:
- Binary cross-entropy as the loss function.
- Adam optimizer with a learning rate of 0.0001.
- Early stopping and learning rate reduction on validation loss.

In [ ]:

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

class Trainer:
    def __init__(self, model, learning_rate=0.0001):
        self.model = model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="binary_crossentropy",
            metrics=["accuracy", tf.keras.metrics.AUC()]
        )

    def train(self, train_dataset, valid_dataset, epochs=20, batch_size=16):
        """Train the model with provided datasets."""
        callbacks = [
            EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
            ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)
        ]

        self.model.fit(
            train_dataset,
            validation_data=valid_dataset,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks
        )


# **4. Evaluation and Saving the Model**

Evaluate the model on a test dataset and save the trained weights.

In [ ]:
    def evaluate(self, test_dataset):
        """Evaluate the model on the test dataset."""
        results = self.model.evaluate(test_dataset)
        print(f"Test Loss: {results[0]:.4f}, Test Accuracy: {results[1]:.4f}, Test AUC: {results[2]:.4f}")

    def save_model(self, path):
        """Save the trained model."""
        self.model.save(path)
        print(f"Model saved to {path}")
